In [2]:
import requests
import json
import pandas as pd
import csv
import time

In [1]:
EMAIL = ""
PASSWORD = ""
URL_LOGIN = "https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/"
headers = {"email": EMAIL,"password":PASSWORD}

In [3]:
response = requests.request("GET", URL_LOGIN, headers = headers)

In [4]:
response_json = json.loads(response.text)

In [5]:
accessToken = response_json["data"][0]["accessToken"]

In [6]:
headers_token = {"accessToken": accessToken}

In [14]:
def load_csv(path):
    with open(path,"r") as input_file:
        reader = csv.reader(input_file,delimiter = ";")
        matrix = [row for row in reader]
    return matrix

In [15]:
def print_csv(file_path, matrix,modificacion):
    with open(file_path, modificacion,newline='') as output_file:
        writer = csv.writer(output_file, delimiter = ";")
        for row in matrix: 
            writer.writerow(row)

## Timetable trips

In [9]:
lineas = load_csv('infoline.csv')

In [10]:
def load_csv(path):
    with open(path,"r") as input_file:
        reader = csv.reader(input_file,delimiter = ";")
        matrix = [row for row in reader]
    return matrix

In [11]:
def print_trips(json):
    return [json["line"],json["date"],json["tripNum"],json["direction"],json["logicBus"],
            json["startTimeTrip"],json["endTimeTrip"],json["dayType"]]

In [16]:
def get_trips(line_id,data_ref):
    data = []
    time.sleep(0.5)
    URL_TIMETABLE_TRIPS = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/" + str(line_id) + "/trips/" + data_ref +"/"
    response_timetable_trips = requests.request("GET", URL_TIMETABLE_TRIPS, headers = headers_token)
    parsed = (json.loads(response_timetable_trips.text)["data"])
    for li in parsed:
        data.append(print_trips(li))
    return data

In [18]:
from datetime import date, timedelta
sdate = date(2018, 2, 1)  
edate = date(2019, 2, 28) 
delta = edate - sdate      
cont = 0
for i in range(delta.days + 1):
    date = sdate + timedelta(days=i)
    fecha = date.strftime('%Y%m%d')
    for i in lineas:
        cont = cont + 1
        print_csv('trips-' + str(cont) +'.csv',get_trips(i[0],fecha),"a")

### Unir todos los conjuntos de datos

In [24]:
from os import listdir
PATH = "trips/"
files = [f for f in listdir(PATH) if "trips" in f]

In [27]:
files_ = []
for f in files:
    df = pd.read_csv(PATH + f, sep=";", header=None)
    files_.append(df)

In [29]:
result = pd.concat(files_)
result.to_csv('trips.csv', sep=';',index=None, header=None)

In [8]:
URL_TIMETABLE_TRIPS = "https://openapi.emtmadrid.es/v1/transport/busemtmad/lines/1/trips/20180203/"
response_timetable_trips = requests.request("GET", URL_TIMETABLE_TRIPS, headers = headers_token)
parsed = (json.loads(response_timetable_trips.text)["data"])

In [9]:
print(parsed)

[{'line': '001', 'date': '03/02/2018', 'tripNum': '0001', 'direction': '2', 'logicBus': '001', 'startTimeTrip': '06:30:00', 'endTimeTrip': '07:05:00', 'dayType': 'SA'}, {'line': '001', 'date': '03/02/2018', 'tripNum': '0002', 'direction': '1', 'logicBus': '001', 'startTimeTrip': '07:05:00', 'endTimeTrip': '07:45:00', 'dayType': 'SA'}, {'line': '001', 'date': '03/02/2018', 'tripNum': '0003', 'direction': '2', 'logicBus': '001', 'startTimeTrip': '07:45:00', 'endTimeTrip': '08:25:00', 'dayType': 'SA'}, {'line': '001', 'date': '03/02/2018', 'tripNum': '0004', 'direction': '1', 'logicBus': '001', 'startTimeTrip': '08:25:00', 'endTimeTrip': '09:17:00', 'dayType': 'SA'}, {'line': '001', 'date': '03/02/2018', 'tripNum': '0005', 'direction': '2', 'logicBus': '001', 'startTimeTrip': '09:17:00', 'endTimeTrip': '10:03:00', 'dayType': 'SA'}, {'line': '001', 'date': '03/02/2018', 'tripNum': '0006', 'direction': '1', 'logicBus': '001', 'startTimeTrip': '10:03:00', 'endTimeTrip': '10:57:00', 'dayType'